<a href="https://colab.research.google.com/github/Venu-GR/INFO-5731_Venu-GR/blob/main/Vennapus_Venu_Gopal_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Tuesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**

**Please check that the link you submitted can be opened and points to the correct assignment.**


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [ ]:
# Your code here

import requests
from bs4 import BeautifulSoup
import pandas as pd

def collect_imdb_reviews(movie_id, max_reviews=1000):
    reviews = []
    page = None
    while len(reviews) < max_reviews:
        url = f"https://www.imdb.com/title/{movie_id}/reviews/_ajax"
        params = {"paginationKey": page} if page else {}
        response = requests.get(url, params=params)
        soup = BeautifulSoup(response.text, 'html.parser')

        review_blocks = soup.find_all('div', class_='text show-more__control')
        reviews.extend([block.text for block in review_blocks])

        if len(reviews) >= max_reviews:
            break

        pagination_key = soup.find('div', {'class': 'load-more-data'})
        if pagination_key:
            page = pagination_key['data-key']
        else:
            break
    return reviews[:max_reviews]



all_reviews = collect_imdb_reviews('tt0120338', 500) +  collect_imdb_reviews('tt1877830', 500)
df = pd.DataFrame({'review': all_reviews})
df.to_csv('imdb_reviews.csv', index=False)
print("Reviews saved to imdb_reviews_optimized.csv")

Reviews saved to imdb_reviews_optimized.csv


# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [ ]:
import re
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
df = pd.read_csv('imdb_reviews.csv')
def clean_text(text):
    text = re.sub(r'[^A-Za-z\s]', '', text)
    text = re.sub(r'\d+', '', text)
    stop_words = set(stopwords.words('english'))
    words = nltk.word_tokenize(text)
    words = [word for word in words if word.lower() not in stop_words]
    words = [word.lower() for word in words]
    ps = PorterStemmer()
    words = [ps.stem(word) for word in words]
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)
df['cleaned_review'] = df['review'].apply(clean_text)
df.to_csv('imdb_cleaned_reviews.csv', index=False)
print("Cleaned reviews saved to imdb_cleaned_reviews.csv")



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Cleaned reviews saved to imdb_cleaned_reviews.csv


# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [ ]:
import spacy
import pandas as pd

df = pd.read_csv('imdb_cleaned_reviews.csv')

nlp = spacy.load("en_core_web_sm")

def pos_analysis(text):
    doc = nlp(text)
    pos_counts = {
        'Nouns': len([token for token in doc if token.pos_ == 'NOUN']),
        'Verbs': len([token for token in doc if token.pos_ == 'VERB']),
        'Adjectives': len([token for token in doc if token.pos_ == 'ADJ']),
        'Adverbs': len([token for token in doc if token.pos_ == 'ADV'])
    }
    return pos_counts

pos_counts = pos_analysis(df['cleaned_review'][0])
print("POS Counts for the first review:", pos_counts)

def parse_sentence(text):
    doc = nlp(text)
    for token in doc:
        print(f"Token: {token.text}, Dependency: {token.dep_}, Head: {token.head.text}")
def named_entity_recognition(text):
    doc = nlp(text)
    for ent in doc.ents:
        print(f"Entity: {ent.text}, Label: {ent.label_}")
example_sentence = df['cleaned_review'][0]
print("\nDependency Parsing for example sentence:")
parse_sentence(example_sentence)
print("\nNamed Entity Recognition for example sentence:")
named_entity_recognition(example_sentence)


POS Counts for the first review: {'Nouns': 37, 'Verbs': 15, 'Adjectives': 19, 'Adverbs': 6}

Dependency Parsing for example sentence:
Token: your, Dependency: poss, Head: millenni
Token: millenni, Dependency: nsubj, Head: attract
Token: like, Dependency: prep, Head: millenni
Token: probabl, Dependency: amod, Head: phenomenon
Token: experienc, Dependency: compound, Head: phenomenon
Token: phenomenon, Dependency: npadvmod, Head: known
Token: known, Dependency: pcomp, Head: like
Token: titanicmania, Dependency: compound, Head: everyon
Token: childhood, Dependency: compound, Head: everyon
Token: everyon, Dependency: dobj, Head: known
Token: ob, Dependency: prep, Head: everyon
Token: movi, Dependency: compound, Head: theatr
Token: theatr, Dependency: pobj, Head: ob
Token: almost, Dependency: advmod, Head: year
Token: year, Dependency: nmod, Head: everywher
Token: everi, Dependency: amod, Head: girl
Token: girl, Dependency: nmod, Head: everywher
Token: ob, Dependency: prep, Head: girl
Token:

#**Comment**
Make sure to submit the cleaned data CSV in the comment section - 10 points

# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

This task was a perfect time to try forward with text scraping, data cleaning and other style of syntactic analysis techniques. Hardest part for this project was the deeper cleansing of text data particularly stemming and lemmatization. I particularly loved trying different styles of parsing and sentence structure. The timeframe allocated was adequate to achieve the task, but it took some extra work in configuring the environment for parsing tools.
.